In [2]:
import plotly.io as pio
pio.renderers.default="iframe"
import pandas as pd
import sqlite3
from plotly import express as px
from sklearn.linear_model import LinearRegression
import seaborn as sns
from matplotlib import pyplot as plt
import requests
import json
import urllib

In [5]:
conn = sqlite3.connect("schools.db") # this creates a database called schools.db.

def query_schools_database(state):
    """
    query_schools_database uses SQL to read through a database containing relevant school district location.

    :state: the name of the state to be investigated
    """
    
    cmd = \
    f"""
    SELECT S.state, S.name, S.lat, S.lon
    FROM schools S
    WHERE S.state = "{state}"
    """
    
    return pd.read_sql_query(cmd, conn)

df2 = query_schools_database("AK")
df2

fig = px.scatter_mapbox(df2, lat='LAT', lon='LON', zoom=1, mapbox_style='carto-positron', hover_name='NAME')
fig.show()

In [60]:
assess_df = pd.read_csv("AlaskaResults.csv")

In [61]:
assess_df.columns = assess_df.iloc[2]

In [62]:
assess_df = assess_df[assess_df["Population"] == "All"]

In [63]:
assess_df = assess_df[assess_df["Grade"] == "All"]

In [64]:
assess_df["District Name"] = assess_df["District Name"] + " School District"

In [65]:
assess_df

2,District ID,District Name,Content Area,Grade,Population,At Target/ Advanced Count,At Target/ Advanced Percentage,Emerging/ Approaching Target Count,Emerging/ Approaching Target Percentage,Enrolled,Participation Rate
3,3,Alaska Gateway School District,ELA,All,All,*,*,*,*,1,100.0%
12,3,Alaska Gateway School District,Math,All,All,*,*,*,*,1,100.0%
21,3,Alaska Gateway School District,Science,All,All,*,*,*,*,2,100.0%
69,4,Aleutian Region School District,ELA,All,All,*,*,*,*,1,100.0%
78,4,Aleutian Region School District,Math,All,All,*,*,*,*,1,100.0%
...,...,...,...,...,...,...,...,...,...,...,...
3806,51,Yukon Flats School District,ELA,All,All,*,*,*,*,2,0.0%
3816,51,Yukon Flats School District,Math,All,All,*,*,*,*,2,0.0%
3826,51,Yukon Flats School District,Science,All,All,*,*,*,*,1,0.0%
3880,54,Yupiit School District,ELA,All,All,*,*,*,*,1,100.0%


In [68]:
assess_df["At Target/ Advanced Percentage"] = assess_df["At Target/ Advanced Percentage"].astype(str).str.replace("*","0")
assess_df["At Target/ Advanced Percentage"] = assess_df["At Target/ Advanced Percentage"].astype(str).str.replace("or more","")
assess_df["At Target/ Advanced Percentage"] = assess_df["At Target/ Advanced Percentage"].astype(str).str.replace("or fewer","")
assess_df["At Target/ Advanced Percentage"] = assess_df["At Target/ Advanced Percentage"].astype(str).str.replace("%","")

In [69]:
assess_df["At Target/ Advanced Percentage"] = assess_df["At Target/ Advanced Percentage"].astype(float)

In [70]:
assess_df["At Target/ Advanced Percentage"] = pd.to_numeric(assess_df["At Target/ Advanced Percentage"], errors="coerce")

average_scores = assess_df.groupby("District Name")["At Target/ Advanced Percentage"].mean().reset_index()

average_scores

,District Name,At Target/ Advanced Percentage
0,Alaska Gateway School District,0.000000
1,Aleutian Region School District,0.000000
2,Anchorage School District,13.500000
3,Annette Island School District,0.000000
4,Bering Strait School District,0.000000
5,Chugach School District,0.000000
6,Denali School District,0.000000
7,Dillingham School District,0.000000
8,Fairbanks School District,16.566667
9,Galena School District,0.000000


In [79]:
average_scores = average_scores.rename(columns={"District Name" : "NAME", "At Target/ Advanced Percentage": "Average Assessment Proficiency"})

In [80]:
alaska_final_df = pd.merge(df2, average_scores, on='NAME', how='left')

In [81]:
alaska_final_df

,STATE,NAME,LAT,LON,Average Assessment Proficiency
0,AK,Lower Kuskokwim School District,60.802916,-161.772970,13.333333
1,AK,Lower Yukon School District,62.092018,-163.724886,0.000000
2,AK,Yupiit School District,60.913883,-161.448992,0.000000
3,AK,Kashunamiut School District,61.531048,-165.585866,NaN
4,AK,Mount Edgecumbe,57.050928,-135.351918,NaN
5,AK,Aleutians East Borough School District,55.342221,-160.496135,NaN
6,AK,Aleutian Region School District,61.173216,-149.882214,0.000000
7,AK,Bering Strait School District,63.876727,-160.793772,0.000000
8,AK,Bristol Bay Borough School District,58.733307,-157.004582,NaN
9,AK,Alaska Gateway School District,63.333206,-142.958861,0.000000


In [85]:
alaska_final_df.to_csv("alaska_df.csv", index=False)